# 06 SemanticKernel | 03 Plugin - Native function

## Native function

Semantic Kernel can import native c# functions and execute them similar to semantic functions (inline defined or imported from file). 

Additional information:
 
- [Overview Native Function](./README.md)
- [Semantic Inline Function](./01_PlugIn_SemanticFunction_Inline.ipynb)
- [Semantic Function from file](./02_PlugIn_SemanticFunction_File.ipynb)

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create Semantic Kernel instance

An instance of Semantic Kernel can be created using the KernelBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with Semantic Kernel functionality or concepts.

It abstracts e.g., models from the OpenAI GPT family and can communicate with Azure OpenAI LLM instances as well as LLMs deployed on OpenAI. In the sample we use models deployed on Azure.

In [5]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using DotNetEnv;
using Microsoft.SemanticKernel;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel initialized...");



Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel initialized...


#### Expected output:

```
Installed Packages
DotNetEnv, 2.5.0
Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel initialized...
```

## Step 2: Define & import native function

A default c# function `RetrieveLocalFile()` is defined in class `NativeFunctions` and registered with Semantic Kernel using the kernel's `ImportFunctions()` function.

In [6]:

using Microsoft.SemanticKernel.Orchestration;
using System.ComponentModel;
using System.IO;

public class NativeFunctions {

    [SKFunction, Description("Retrieve content from local file")]
    public async Task<string> RetrieveLocalFile(string fileName, int maxSize = 5000) 
    {
        string content = await File.ReadAllTextAsync(fileName); 
        if (content.Length <= maxSize) return content;
        return content.Substring(0, maxSize);

    }
}

//Import native function
string plugInName = "NativeFunction";
string functionName = "RetrieveLocalFile";

NativeFunctions nativeFunctions = new NativeFunctions();
kernel.ImportFunctions(nativeFunctions, plugInName);

Console.WriteLine($"Inline function {functionName} defined and imported...");


Inline function RetrieveLocalFile defined and imported...


#### Expected output:

```
Inline function RetrieveLocalFile defined and imported...
```

## Step 4: Execute native function

Semantic Kernel doesn't make a difference if a function is a semantic function (defined inline or imported from directory) or if it is a native function. Therefore, calling the native function isn't different from calling semantic functions.

The kernel function `RunAsync()` will be used to execute the native function.

In [7]:
string assetsFolder = @"../../assets";
string fileName = Path.Combine(assetsFolder,"docs","07_SemanticKernel", "aci_documentation.txt");

ContextVariables contextVariables = new ContextVariables();
contextVariables.Add("fileName", fileName);
contextVariables.Add("maxSize", "5000");

ISKFunction skFunction = kernel.Functions.GetFunction(plugInName, functionName);

KernelResult kernelResult = await kernel.RunAsync(contextVariables, skFunction);         

string wikiArticle = kernelResult.GetValue<string>() ?? "";

Console.WriteLine($"Wiki article: {wikiArticle.Substring(0,50)}...");


Wiki article: Configure a GitHub Action to create a container in...


#### Expected output:

```
Wiki article: Configure a GitHub Action to create a container in...
```

## Next steps

Now that semantic functions and native functions have been defined, registered and executed next steps are:

- [Explore Memories (Overview)](README.md)
- [See Memories in action](./04_Memory.ipynb)

